# Task 1: Retrieving and Preparing the Data

In [1]:
import datetime as dt

# Importing packages - Pandas, Numpy, Seaborn, Scipy
import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns, sys
import matplotlib.style as style; style.use('fivethirtyeight')
from scipy.stats import zscore, norm

np.random.seed(0)

# Modelling - LR, KNN, NB, Metrics
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

# Oversampling
from imblearn.over_sampling import SMOTE

# Suppress warnings
import warnings; warnings.filterwarnings('ignore')
pd.options.display.max_rows = 4000

In [ ]:
file_path = "./online_retail_II.xlsx"
df = pd.read_excel(file_path)

## Data Shape

## Attribute Information

* **`Invoice`**: Invoice number. Nominal. A 6-digit integral number uniquely assigned to each transaction. If this code starts with the letter 'c', it indicates a cancellation.
* **`StockCode`**: Product (item) code. Nominal. A 5-digit integral number uniquely assigned to each distinct product.
* **`Description`** : Product (item) name. Nominal.
* **`Quantity`**: The quantities of each product (item) per transaction. Numeric.
* **`InvoiceDate`**: Invice date and time. Numeric. The day and time when a transaction was generated.
* **`Price`**: Unit price. Numeric. Product price per unit in sterling (£).
* **`Customer ID`**: Customer number. Nominal. A 5-digit integral number uniquely assigned to each customer.
* **`Country`**: Country name. Nominal. The name of the country where a customer resides.

In [ ]:
df.info()

## Data Cleaning
**Missing values**

In [ ]:
df.isnull().any()

In [ ]:
print('Number of invoices for cancelation that also have negative quantity',
      df.loc[(df['Invoice'].str.contains('C', na = False)) & (df['Price'] < 0)].shape[0])

Dataset has 525461 rows and 8 columns, with missing values in **Customer ID** and **Description** columns.

In [ ]:
df.isnull().sum()

df.isnull().sum()


In specific, the **Description** column has 2928 missing values while **CustomerID** column has 107927 null values.

In [ ]:
df.duplicated().sum()

In [ ]:
df[df["Invoice"].str.contains("C", na=False)]

**Description:**

In [ ]:
df[df["Description"].isnull() == True].head()

Those columns having null values in **Description, Customer ID** and **Price** equals 0 are dropped because they are considered failed transactions.

In [ ]:
# Drop failed transactions
idx = df[(df['Description'].isnull()) & (df['Customer ID'].isnull()) & (df['Price']==0)].index.values
df.drop(idx, inplace=True)

In [ ]:
# Check the missing values
# Feature Engineering
print(df.isnull().any().sum()," are dropped")

**Quantity:**
The quantity column not only displays the quantity of items purchased, but it also displays the amount of cancelled/returned items by encoding the cancelled transactions as negative. Each of these cancelled transactions is associated with the Invoice Number. However, there is only 1 instance whose invoice starts with C but the Quantity is positive (1).

In [ ]:
print('The number of entries with negative quantity', df[(df['Quantity'] < 0)].shape[0])

In [ ]:
print('Number of invoices for cancelation that also have negative quantity',
      df.loc[(df['Invoice'].str.contains('C', na = False)) & (df['Quantity'] < 0)].shape[0])

In [ ]:
df.loc[(df['Invoice'].str.contains('C', na = False)) & (df['Quantity'] < 0)]

In [ ]:
df.loc[(df['Invoice'].str.contains('C', na = False)) & (df['Quantity'] >= 0)]

In [ ]:
import matplotlib.pyplot as plt

df[["Price","Quantity"]].plot(kind="box")

**InvoiceDate**

In [ ]:
max_invoice_date = df['InvoiceDate'].max()
min_invoice_date = df['InvoiceDate'].min()
date_fmt = '%Y-%m-%d'
print('The data ranges from {} to {}'.format(min_invoice_date.strftime(date_fmt), 
                                             max_invoice_date.strftime(date_fmt) 
                                            )
     )

In [ ]:
duplicated = df[df.duplicated(keep = False)].sort_values(by = ['InvoiceDate','Invoice','StockCode'])
duplicated

## Add new columns

In [ ]:
# Add new column: OrderCancelled
def map_order_cancelled(row):
    return 1 if "C" not in str(row).upper() else 0
    
df["OrderCancelled"] = df["Invoice"].map(map_order_cancelled)

In [ ]:
def map_international_shipping(row):
    return 1 if row == "United Kingdom" else 0
df['International Shipping']= df['Country'].map(map_international_shipping)
df['International Shipping'].value_counts()

In [ ]:
# Add new column: TotalPrice
df["TotalPrice"] = df["Quantity"] * df["Price"]

In [ ]:
# Add date related columns
df['Week']=df['InvoiceDate'].dt.week
df["Week Day"] = df["InvoiceDate"].dt.weekday
df['Day']=df['InvoiceDate'].dt.day
df['Month']=df['InvoiceDate'].dt.month
df["Quarter"] = df["InvoiceDate"].dt.quarter
df["Year"] = df["InvoiceDate"].dt.year

def map_is_weekend(row):
    return 1 if row == 5 or row == 6 else 0
df['isWeekend']= df['Week Day'].map(map_is_weekend)

# Task 2: Feature Engineering

In [ ]:
corr = df.drop('OrderCancelled', axis = 1).corr()

mask = np.zeros_like(corr, dtype = np.bool)
mask[np.triu_indices_from(mask)] = True

f, ax = plt.subplots(figsize = (11, 9))

cmap = sns.diverging_palette(220, 10, as_cmap = True)

sns.heatmap(corr, mask = mask, cmap = cmap, vmax = 1, center = 0, square = True, 
            linewidths = .5, cbar_kws = {"shrink": .5}, annot = True, fmt="0.2f")
ax.set_title('Correlation Matrix of Data')